In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/umranyaman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
#https://www.kaggle.com/uciml/sms-spam-collection-dataset
df_train = pd.read_csv('spam_train.csv', encoding = 'ISO-8859-1', header=0)
df_train.head()

,sms,category
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [45]:
df_test = pd.read_csv('spam_test.csv', encoding = 'ISO-8859-1', header=None)
df_test = df_test.drop(columns=[2])
df_test = df_test.rename(columns={0:"sms", 1:"category"})
df_test = df_test.drop([0])
df_test.head()

,sms,category
1,Desires- u going to doctor 4 liver. And get a ...,ham
2,Hmmm.still we dont have opener?,ham
3,Yeah so basically any time next week you can g...,ham
4,"Edison has rightly said, \A fool can ask more ...",ham
5,I will vote for wherever my heart guides me,ham


In [46]:
tokenizer = RegexpTokenizer('r\w+')
stopwords_english = set(stopwords.words('english'))

#Tokenizing & steamming & removing stop words
def cleanSms(sms):
    sms=sms.replace("<br /><br />"," ")
    sms=sms.lower()
    sms_tokens=tokenizer.tokenize(sms)
    sms_tokens_without_stopwords=[token for token in sms_tokens if token not in stopwords_english ]
    stemmed_sms_tokens_without_stopwords=[PorterStemmer().stem(token) for token in sms_tokens if token not in stopwords_english]
    cleaned_sms = ' '.join(stemmed_sms_tokens_without_stopwords)
    return cleaned_sms

In [48]:
#Clean the data & plot it on X & Y 
df_train['sms'].apply(cleanSms)
x_train=df_train['sms'].values
y_train=df_train['category'].values

df_test['sms'].apply(cleanSms)
x_text=df_test['sms'].values
y_text=df_test['category'].values

In [50]:
#Vectorize the data
from sklearn.feature_extraction.text import TfidVectorizer
vectorizer = TfidVectorizer(sublinear_tf=True, encoding='ISO-8859-1')
vectorizer.fit(x_train)
x_train=vectorizer.transform(x_train)
x_test=vectorizer.transform(x_test)

ImportError: cannot import name 'TfidVectorizer' from 'sklearn.feature_extraction.text' (/Users/umranyaman/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py)

In [51]:
#Create model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="lbgfs")
model.fit(x_train,y_train)

ValueError: Logistic Regression supports only solvers in ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'], got lbgfs.

In [ ]:
#Predict Spam
model.predict(vectorizer.transform(['you won $900 in the new lottery draw. Call 12345']))

In [ ]:
#Predict Ham
model.predict(vectorizer.transform(['Hello there. How are you doing?']))

# Pickle Operation

In [ ]:
import joblib
joblib.dump(model, 'spam_ham_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')